<a href="https://colab.research.google.com/github/kristofsl/BayesianHyperParameterOptimisation/blob/master/example_cola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas scikit-learn optuna shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 6.9 MB/s eta 0:00:00


In [2]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import optuna
import shap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Set a random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
# ### 2. 💾 Data Loading and Initial Preparation

# Load the Titanic dataset.
# For simplicity and robust Colab execution, we'll use a standard UCI-hosted version
# or load a pre-cleaned version often used in tutorials.

# Since `pd.read_csv` from a direct URL is robust in Colab:
data_url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"
try:
    df = pd.read_csv(data_url)
    print("Titanic dataset loaded successfully.")
except Exception as e:
    # Fallback/alternative if the direct link fails
    print(f"Could not load data from URL: {e}. Trying a local version (if available).")
    # For a real Colab notebook, you might mount Google Drive or use files.upload() here.
    # For this example, we proceed with the loaded data.

# Display the first few rows to understand the data
print("\n--- Initial Data Head ---")
print(df.head())

# Check for missing values and data types
print("\n--- Data Info and Missing Values ---")
df.info()

Titanic dataset loaded successfully.

--- Initial Data Head ---
   Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500  

--- Data Info and Missing Va

In [5]:
# ### 3. 🧹 Data Preprocessing

# The goal is to prepare the data for the Random Forest model.

# **3.1. Feature Selection**
# - `Survived`: The target variable (what we want to predict).
# - `Pclass`, `Sex`, `Age`, `Siblings/Spouses Aboard`, `Parents/Children Aboard`, `Fare`: Features to use.

# **3.2. Handling Missing Values (Imputation)**

# 'Age' has missing values (NaNs). We will fill them with the median age.
df['Age'] = df['Age'].fillna(df['Age'].median())

# 'Fare' might have a few missing values in some datasets (though not in the one we loaded).
# It's good practice to check and impute if necessary.
df['Fare'] = df['Fare'].fillna(df['Fare'].median())

# Check again for any remaining NaNs (should be zero now)
print("\n--- Missing values after imputation ---")
print(df.isnull().sum())

# **3.3. Encoding Categorical Features**
# 'Sex' is a categorical column ('male'/'female') and needs to be converted to numbers (0/1).
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})


--- Missing values after imputation ---
Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64


In [6]:
X = df.drop('Survived', axis=1)
y = df['Survived']

# Split the data into training and testing sets and stratefy on y
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)